In [6]:
def templates():
	""" store all prompts templates """
	behavioral_template = """ I want you to act as an interviewer. Remember, you are the interviewer not the candidate.   
            Let's think step by step.
            
            Based on the keywords, 
            Create a guideline with the following topics for a behavioral interview to test the soft skills of the candidate. 
            
            Do not ask the same question.
            Do not repeat the question. 
            
            Keywords: 
            {context}
       
            Question: {question}
            Answer:"""
	conversation_template = """I want you to act as an interviewer strictly following the guideline in the current conversation.
                            Candidate has no idea what the guideline is.
                            Ask me questions and wait for my answers. Do not write explanations.
                            Ask each question like a real person, only one question at a time.
                            Do not ask the same question.
                            Do not repeat the question.
                            Do ask follow-up questions if necessary. 
                            Your name is GPTInterviewer.
                            I want you to only reply as an interviewer.
                            Do not write all the conversation at once.
                            If there is an error, point it out.

                            Current Conversation:
                            {history}

                            Candidate: {input}
                            AI: """

In [32]:
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import SpacyTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.chains import (
    StuffDocumentsChain, LLMChain, ConversationalRetrievalChain
)
def embeddings(text):
	text_splitter = SpacyTextSplitter()
	text_splitter = SpacyTextSplitter(pipeline="en_core_web_sm")
	texts = text_splitter.split_text(text)
	embeddings = OpenAIEmbeddings()
	docsearch = FAISS.from_texts(texts, embeddings)
	retriever = docsearch.similarity_search(text)
	return retriever

In [33]:
from dataclasses import dataclass
from typing import Literal
from langchain_openai import ChatOpenAI
from langchain.llms import OpenAI
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
load_dotenv()
import os
openai_api_key = os.getenv("OPENAI_API_KEY")
@dataclass
class Message:
	origin: Literal["human", "ai"]
	message: str

In [34]:
# %%time
from langchain.globals import set_llm_cache

from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
def load_llm():
    # Load the locally downloaded model here
    llm = OpenAI(
        model_name="gpt-3.5-turbo-instruct",
        max_new_tokens=50,
        temperature=0.5
    )
    return llm
# llm = load_llm()
# prompt = templates()
# Retrieval QA Chain
def retrieval_qa_chain(llm, prompt, db):
#     qa_chain = RetrievalQA.from_chain_type(llm=llm,
#                                            chain_type='stuff',
#                                            retriever=db.as_retriever(search_kwargs={'k': 2}),
#                                            return_source_documents=True,
#                                            chain_type_kwargs={'prompt': prompt}
#                                            )
#     template = (
#     "Combine the chat history and follow up question into "
#     "a standalone question. Chat History: {chat_history}"
#     "Follow up question: {question}"
# )
#     prompt = PromptTemplate.from_template(template)
    # retriever=db.as_retriever()
    question_generator_chain = LLMChain(llm=llm, prompt=prompt)
    qa_chain = ConversationalRetrievalChain(
        # retriever=retriever,
        question_generator=question_generator_chain,
    )
    return qa_chain

# QA Model Function
def qa_bot(text):
    db = embeddings(text)
    llm = load_llm()
    qa_prompt = templates()
    qa = retrieval_qa_chain(llm, qa_prompt, db)

    return qa

# Output function
def final_result(query):
    qa_result = qa_bot(query)
    response = qa_result({'query': query})
    return response

# Behavioral_Prompt = PromptTemplate(input_variables=["context", "question"],
#                                           template=templates.behavioral_template)
job = "Obvious Technology Inc. is a cognitive enterprise platform that simplifies AI for businesses. We use deep technology, including computer vision, natural language processing, and machine learning, to build a cognitive platform powered by our proprietary AiBlock™. Our goal is to provide affordable AI solutions to our clients in industries such as healthcare, banking, insurance, financial services, manufacturing, and retail. With 11 patents, 24 proprietary AiBlocks™, and over 60 business use cases developed, we are dedicated to making AI accessible to all"
print(final_result(job))

/Users/amallmuhammed/Documents/Bro Project/Intelligent-HR/project/lib/python3.11/site-packages/spacy/pipeline/lemmatizer.py:211: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)
/Users/amallmuhammed/Documents/Bro Project/Intelligent-HR/project/lib/python3.11/site-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! max_new_tokens is not default parameter.
                max_new_tokens was transferred to model_kwargs.
                Please confirm that max_new_tokens is what you intended.
  warnings.warn(


ValidationError: 1 validation error for LLMChain
prompt
  value is not a valid dict (type=type_error.dict)

In [ ]:
from langchain.memory import 